# Information for groups working on project 8: the Atlantic Multidecadal Oscillation in CMIP6 models

---



The aim of this notebook is to give you some information that might be useful for starting your project:

- Part A provides you with a quick introduction to the CMIP6 project and ensemble modelling. It also contains a list of links that provide details about the data that are available to you in the CMIP6 catalogue. You can use this to help you to decide what data you want to analyse for your project.
- Part B provides some examples of how to plot data on maps easily, by combining xarray (which you saw in Guillaume's Pangeo session) with Cartopy
- Part C gives you more specific information about this project, about the Atlantic Multidecadal Oscillation (AMO), and about what you should be aiming to do. You will need to calculate specific time series and patterns in order to characterise the AMO in different models. Some ideas for the directions that you might take after that are given at the end of the notebook.

Note: the installation of the libraries to plot maps using Cartopy can take a few minutes. You might want to execute the first code cell of part B now, before you read the information below, so that you don't have to wait later.

## A. Data sources and information about CMIP6

### 1. Introduction
Project 8 requires you to analyse model output from the CMIP6 project. It is possible to access these data a number of ways, including via the cloud, since Google host a part of these data via their Public Datasets programme (https://cloud.google.com/blog/products/data-analytics/new-climate-model-data-now-google-public-datasets). For your project, this is how you will access the CMIP6 data. As you have seen in class, this method means that you will be able to analyse the data directly in Colab without downloading anything to your machine.

### 2. What is ensemble modelling and why do we do it?
CMIP6 uses an ensemble modelling strategy. What does this mean?

In the context of CMIP6, an ensemble of model simulations is a collection of simulations that have been created using exactly the same configuration for everything, with the exception of the initial conditions. These small differences in the initial state can lead to much larger differences in the forecast over time, as popularised in the so-called "butterfly effect".

We use ensemble modelling to help us understand the probability of the events simulated by the model occurring. For example, imagine that we use an ensemble of 5 simulations to try to forecast next week's weather, and all 5 simulations predict that it will rain in Brest. In this case, we will be more likely to believe that it will rain next week than if our results had been that 2 of our simulations told us that it would rain, 2 told us that it would be sunny and 1 told us that it would snow.

The aim of the ensemble approach in climate modelling (as in CMIP6) is the same as in the weather forecasting case: it is an attempt to quantify how much we believe the predictions of the future climate.

### 3. What is CMIP6?
CMIP = Coupled Model Intercomparison Project (wikipedia page: https://en.wikipedia.org/wiki/Coupled_Model_Intercomparison_Project)

The CMIP programme aims to improve our knowledge of climate change. It began in 1995, and has already been through 5 "phases". Each "phase" corresponds to a coordinated effort to collect together a number of climate simulations created by research centres around the world. These simulations are then made freely available for anybody to download and analyse. Phase 6 is now underway: the data that you will be analysing are thus only recently available, and are likely to be used in research studies for about the next ten years or so.

A number of experiments are defined at each phase. These often consist of "historical" experiments, which simulate the climate over the past decade, and a number of future experiments, which simulate the future climate in response to certain prescribed greenhouse gas concentration scenarios, which should represent the response to possible future decisions made by society. (If you are interested in reading more about these, this article provides a nice introduction: https://www.carbonbrief.org/explainer-how-shared-socioeconomic-pathways-explore-future-climate-change)

### 4. What data are available for your projects?
The CMIP6 catalogue contains a lot of data: the entire archive comprises about 20PB of outputs! Not all available data is stored in the Cloud, so you will need to check to see what is available when deciding which model(s) and scenario(s) you want to analyse.

The people at Pangeo have written some useful summaries that describe:

- the data available in the Cloud: https://docs.google.com/document/d/1yUx6jr9EdedCOLd--CPdTfGDwEwzPpCF6p1jRmqx-0Q/edit
- a list of the models available: https://docs.google.com/spreadsheets/d/13DHeTEH_8G08vxTMX1Fs-WbAA6SamBjDdh0fextdcGE/edit#gid=165882553
- a list of the variables available: https://docs.google.com/spreadsheets/d/1UUtoz6Ofyjlpx5LdqhKcwHFz2SGoTQV2_yekHyMfL9Y/edit#gid=1221485271

They have also produced an example notebook that shows how to load in air temperature. You can either run it via Binder on the Pangeo infrastructure: https://binder.pangeo.io/v2/gh/pangeo-data/pangeo-cmip6-examples/master or run it directly in Colab: https://colab.research.google.com/drive/19iEVxE_9QoTeg4st7MmucHJUmO93NXHp

Their notebook is an excellent introduction: it should provide you with everything that you need to get started loading the data. I recommend that you look at it!

## B. Using xarray with Cartopy

Last week we talked a lot about accessing and analysing data, but we didn't talk much about how to make figures. In the geosciences, often we want to make maps. Unfortunately, sometimes this can be more difficult than we would like! Model grids are often irregular, and it can be difficult to plot data in the polar regions, where all longitudes get closer and closer together, and finally converge to a single point.

The Pangeo notebook shows an example of loading CMIP6 data using xarray. One of the really nice features of xarray is that it integrates with the plotting library Cartopy (https://scitools.org.uk/cartopy/docs/latest/) to enable us to change map projections and easily transform our data. We'll look at an example for the Arctic below.

Let's load in the sea surface height for the IPSL model in the Shared Socioeconomic Pathway 2 scenario:

In [ ]:
# first we have to install some libraries in Colab
!pip install zarr
!pip install gcsfs

# and set up Cartopy
!pip uninstall shapely --yes

# note: the next step produces an error in pip. You can ignore this error.
!pip install lida==0.0.10 shapely cartopy --no-binary shapely --no-binary cartopy --use-deprecated=legacy-resolver

# and manually download shape files for Cartopy
!wget https://raw.githubusercontent.com/SciTools/cartopy/main/lib/cartopy/feature/download/__main__.py -O cartopy_feature_download.py
!python cartopy_feature_download.py physical

In [ ]:
# set up our libraries
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs

# set up cartopy
import cartopy.crs as ccrs
from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
GeoAxes._pcolormesh_patched = Axes.pcolormesh

# notebook display options
xr.set_options(display_style='html')
%matplotlib inline

Now let's look at the available data in the catalogue, and select one of the simulations:

In [ ]:
# read in the catalogue information
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')

# look for simulations that correspond to our chosen criteria
df_ssh = df.query("activity_id=='ScenarioMIP' & table_id == 'Omon' & variable_id == 'zos' & experiment_id == 'ssp245' & institution_id == 'IPSL'")

# print out the metadata for the available simulations in the catalogue
df_ssh

The table shows multiple available outputs for the same forcing scenario and model: these are the different members (member_id column) of the ensemble for that model and scenario. Now let's select one of the members:

In [ ]:
# setup the gcs access
gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the last one from the dataframe above)
zstore = df_ssh.zstore.values[-1]

# create a mapping interface to the store
mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
ds = xr.open_zarr(mapper, consolidated=True,decode_times=False)

The name given to the latitude and longitude variables changes depending on the model, so let's check the names of the coordinate variables for this model:

In [ ]:
ds.coords

Latitude is called ```nav_lat``` in this model, and longitude is called ```nav_lon```. Now we'll select only the North Atlantic region. We'll choose the region 120°W to 30°E, 0 to 65°N:

In [ ]:
lat = ds.nav_lat.compute()
lon = ds.nav_lon.compute()
ds = ds.where(((lat > 0) & (lat < 65) & (lon > -120) & (lon < 30)),drop=True)

Now let's see what happens if we try to plot a map for a specific date

In [ ]:
ds.zos.sel(time=ds.time[-1]).squeeze().plot()

It's not so easy to see what's going on with this simple map. We can make the map easier to read by choosing a [projection](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html#cartopy-projections), centering the map on the North Atlantic region, and adding some coastlines. Without adding any data, our map will then look like this:

In [ ]:
fig = plt.figure()
# select the projection that we want
ax = plt.axes(projection=ccrs.Mercator());
# then we set the latitude and longitude limits on our figure
# ([west_longitude,east_longitude,south_latitude,north_latitude])
ax.set_extent([-110, 20, 5, 65], ccrs.PlateCarree())
# and add the coastlines
ax.coastlines()

To be able to plot our data on this map, we also need to apply a transformation. We can just tell xarray to do this for us, by telling it the name of the transformation to apply:

In [ ]:
# first we set up our axes with the projection that we want
ax = plt.axes(projection=ccrs.Mercator());
# then we set the latitude and longitude limits on our figure
# ([west_longitude,east_longitude,south_latitude,north_latitude])
ax.set_extent([-100, 20, 5, 65], ccrs.PlateCarree())
# now we tell xarray to plot our data. We pass a number of extra parameters:
# transform=ccrs.PlateCarree() -> tells xarray how to transform the data
# x='nav_lon' -> tells xarray the name of the longitude variable
# y='nav_lat' -> tells xarray the name of the latitude variable
ds.zos[0].plot.pcolormesh(ax=ax,transform=ccrs.PlateCarree(),x='nav_lon', y='nav_lat', add_colorbar=True);
# finally, we add the coastlines
ax.coastlines();
# and add grid lines
gl = ax.gridlines(draw_labels=True, x_inline=False, y_inline=False, color='grey')
# by default, labels are placed on both the right and left hand sides of the
# figure, and at both the top and bottom. The commands below turn off the
# labels on the right hand side, and on the top
gl.right_labels = False
gl.top_labels = False

## C. Aims for your project

In this project, you should investigate how the Atlantic Multidecadal Oscillation (AMO) is represented in different models in the CMIP6 experiment.

The AMO is a low-frequency oscillation of "natural" variability, that is associated with large-scale, multidecadal changes in sea surface temperature. You can find an introduction to the AMO here: [https://climatedataguide.ucar.edu/climate-data/atlantic-multi-decadal-oscillation-amo](https://climatedataguide.ucar.edu/climate-data/atlantic-multi-decadal-oscillation-amo).

You will see in the figures on this page that the AMO is characterised by a time series index, that describes the strength of the oscillation over time, and also by a map, that describes the relative strength as a function of position. You will calculate both the index and the map for each model, to describe the AMO.

### Choosing your data:

For your project, you can choose to either:

*   Compare different models in the same forcing scenario

or:

*   Choose a single model, and compare different ensemble members and different forcing scenarios

Once you have chosen which of these two options you want to work on, the first thing that you will need to do is to identify the available models in the catalogue.

The models that you choose will need to have sea surface temperature (SST) data available, and also the grid-cell area for ocean variables. You should check for this when you query the catalogue.

Some models will also have the global average sea surface temperature available: your analysis will be easier to perform if this is available for the models that you choose.

### First step: defining the AMO index
Once you have identified the available models, you will then need to calculate the AMO index for each of the models that you have chosen. This is done in multiple steps:
1. Calculate the area-weighted sea surface temperature average over the region 0-60°N, 0-80°W. This gives you a time series of the average SST in the North Atlantic. Remove the seasonal cycle from this time series to obtain the sea surface temperature anomaly (SSTA).
2. Either use the model-supplied value (if available) of the global, area-weighted SST average or, if not available, calculate it. This gives you a time series that characterises the overall global warming signal in the ocean. Remove the seasonal cycle from this time series also, to obtain the SSTA. Note that you will also need this to calculate the AMO pattern in the second part of the analysis below, so if you are not using a model-supplied value, you might wish to save the results of your calculation here.
3. Subtract the global average time series from your North Atlantic SSTA average time series. This will remove the global warming signal from your estimate in the North Atlantic, and leave you with just the "natural" part of the variability.
4. Finally, you should smooth the data using a 10-year low-pass filter (for example, a rolling mean. You can do this using the ```rolling``` method in xarray). This time series now corresponds to the AMO index as it is usually defined (e.g. [Trenberth and Shea, 2006](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2006GL026894)).

### Second step: calculating the AMO pattern
Calculation of the AMO pattern is also done in multiple steps, in order to remove the global SST changes associated with global warming. You will need to perform the following steps:
1. You will need the North Atlantic region SST data (0-60°N, 0-80°W again), and the global, area-weighted SSTA average used in step 2 when you calculated the AMO index above. Remove the seasonal cycle from the data, and then perform a linear regression to fit the North Atlantic SSTA data to the global SSTA time series. You should obtain a map of coefficients as the output. You can perform this step using xarray's ```polyfit``` function, but it requires a little work - an example is shown below.
2. Use the coefficients that you have just calculated to produce estimates of the SST at each point as a function of the global SSTA time series. Again, you can use xarray for this, with the ```polyval``` function, and an example is shown below to help you with this step. This estimate represents the local imprint of the global SST change as a function of time.
3. Subtract this estimate from the original North Atlantic region SSTA data. This removes the signature of the global SSTA change at each point, and leaves you with an SSTA anomaly relative to the global SSTA change at each point.
4. Using the same approach as in step 1, perform a linear regression to fit your anomaly data from step 3 to the AMO index that you calculated previously. The map of coefficients that you finally obtain is the AMO pattern. It corresponds to the model's version of the map that you can see on the Climate Data Guide page, and represents the spatial signature of the AMO for that model.

Once you have performed these two sets of calculations, you have characterised the AMO for the model that you have chosen. You should have a time series (the AMO index), and a map of coefficients (the AMO pattern). If you repeat the analysis for different models, the indices and patterns that you obtain may resemble one another, or may be quite different.

In [ ]:
# An example of how to use xarray's polyfit and polyval functions to perform the regression
# of the North Atlantic region SST data against the global mean SST:
import numpy as np

# ==============================
# === first choose some data ===
# ==============================
# read in the catalogue information
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')

# look for simulations that correspond to our chosen criteria and contain SST data
df_sst = df.query("activity_id=='ScenarioMIP' & table_id == 'Omon' & variable_id == 'tos' & experiment_id == 'ssp245' & institution_id == 'IPSL'")

# look for simulations that correspond to our chosen criteria and contain global mean SST data
df_sst_g = df.query("activity_id=='ScenarioMIP' & table_id == 'Omon' & variable_id == 'tosga' & experiment_id == 'ssp245' & institution_id == 'IPSL'")

# get the list of members for which both variables are available
members = np.union1d(df_sst.member_id.values,df_sst_g.member_id.values)

# and take the first one in the list to use in this example
target_member = members[0]
df_sst = df_sst.loc[df_sst['member_id'] == target_member]
df_sst_g = df_sst_g.loc[df_sst_g['member_id'] == target_member]

# === now we retrieve these two variables
gcs = gcsfs.GCSFileSystem(token='anon')
# first retrieve SST
mapper = gcs.get_mapper(df_sst.zstore.values[0])
ds_sst = xr.open_zarr(mapper, consolidated=True)
# then global SST
mapper = gcs.get_mapper(df_sst_g.zstore.values[0])
ds_sst_g = xr.open_zarr(mapper, consolidated=True)

# and reduce the SST to the North Atlantic region:
lat = ds_sst.nav_lat.compute()
lon = ds_sst.nav_lon.compute()
ds_sst = ds_sst.where(((lat > 0) & (lat < 60) & (lon > -80) & (lon < 0)),drop=True)

# ==================================
# === now perform the regression ===
# ==================================
# === Note, the example below shows how to perform the regression, but does
# === not remove the seasonal cycle from either the tos or the tosga data.
# === Don't forget to perform this step in your analyses!

# the xarray polyfit method only works on "dimension" variables. In our case,
# this means that we could only use it for the two spatial dimensions (x,y), or
# for time. But this is not what we want to do: instead we want to be able to
# predict a value of local SSTA given a value of the global SSTA. So we
# need to create a new xarray Dataset, where the dimension variable is the
# variable that we want to use to make our predictions (the global SST).
# Let's make the new data set:
ds_regression = xr.Dataset({'tos': (('tosga','y','x'),ds_sst.tos.values)},
                 coords={'tosga': ('tosga',np.ravel(ds_sst_g.tosga.values))})
# this contains the North Atlantic SST data as a data variable, and the global
# SST data as a dimension variable. Now we can perform the linear regression:
sst_coeffs = ds_regression.polyfit('tosga',1)
# -- we have now completed the first part of step 2.

# next we want to predict the North Atlantic SST at each point as a function
# of the global SST using these coefficients. We can do this using xr.polyval:
tfit = xr.polyval(ds_regression.tosga,sst_coeffs)
# and then we rename the dimensions of these outputs to be consistent
# with the original data set:
tfit = tfit.rename_dims({'tosga': 'time'})
# note: the above step produces a warning. You can ignore it.

# replace the global time series data that we used for the regression by
# the original time data
tfit['tosga'] = ('time',ds['time'].values)
# and also rename the variables
tfit = tfit.rename({'tos_polyfit_coefficients': 'tos', 'tosga': 'time'})
# -- we have now completed the second part of step 2

# Finally, we can subtract this estimate from the original data. The dimensions
# and variable names are all the same, so xarray should do this for us with no
# problems:
ds_sst['tos'] = ds_sst['tos'] - tfit['tos']
# -- we have now completed the third part of step 3: we have removed the
# imprint of the global SST change from each point in the grid, and we are
# left with the SST anomaly.

# By modifying and repeating the regression example above, you can
# calculate the regression of the SST anomaly data on to your AMO index
# to obtain the AMO pattern

## What next?
Once you have calculated your patterns and AMO time series you can compare them. Do you see differences in the periodicities or in the spatial patterns?

To go further with your analyses you might choose to look at one of the following:

*   Can you find any links with the North Atlantic Oscillation? The North Atlantic Oscillation can be defined as the first mode of the sea level pressure in winter (DJF) using a principal component analysis. You can calculate this for the models that you have chosen and see whether you can find any covariability between the AMO and the NAO.
*   Finding common features in the models. Can you find ways to group the models into different categories of common behaviour using some of the methods that we have seen in the course? For example: can you identify the scenario that a given time series represents by using a classification mechanism? Or can you use clustering to identify common patterns in the spatial AMO patterns or the AMO index?
*   Predicting the AMO. Can you predict the behaviour of the last third of the AMO time series given the earlier part of the time series? Or predict how a model will behave in one forcing scenario based on a different forcing scenario?
